## 需求

现在我的需求是要获取图中多个成功的buildid信息，哪种是成功的buildid呢？就是有标记`icon-blue`的。

![](./buildid_rquirement.png)

## 使用requests

首先尝试的是使用`requests`，但是通过它获取到的网页信息太有限了（内容见requests_output.txt），这是因为该网页包含了太多的javascript脚本，这些脚本使得只有在真正的浏览该网页时一些必要的信息才会加载。

In [4]:
import requests

url = 'https://ece-ci.dynamic.nsn-net.net/job/MASTER/job/GNB/job/UPLANE/job/L2-LO/job/SCT.fuse.asib_abio/buildTimeTrend'
r = requests.get(url, verify=False)

with open("requests_output.txt", "w", encoding="utf-8") as f:
    f.write(r.text)

c:\Users\lianbche\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ece-ci.dynamic.nsn-net.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
